In [6]:
import pandas as pd
import numpy as np
import re # используется для кодирования Балконов, там числа со строками и знаками вперемешку

In [7]:
#Reading CSV file
df = pd.read_csv('../data/_data.csv')
pd.set_option('display.max_columns', None)

In [8]:
#Deleting Unnecessary columns from data frame
df = df.drop(['Unnamed: 0', 'Тип'], axis=1)
df = df.drop(['Парковка'], axis=1) # Убераем парковку
df = df.drop(['Телефоны'], axis=1) # Убераем телефоны
df = df.drop(['Площадь комнат, м2'], axis=1) # Убераем площади комнат

In [9]:
#Renaming columns from Russian to English
df = df.rename(columns={'ID  объявления': 'post_id', 'Количество комнат': 'number_of_rooms', 
                            'Метро': 'metro', 'Адрес': 'address', 'Площадь, м2': 'area_m2', 'Дом': 'apt_short_desc'})

In [10]:
#Replacing nan values by the most frequent element 
df['number_of_rooms'] = df['number_of_rooms'].replace(to_replace=[None], value=df.loc[:, 'number_of_rooms'].mode()[0])

In [11]:
#Creating 3 new features in data frame
df.insert(2, 'isolated_apt', 0)
df.insert(3, 'adjacent_apt', 0)
df.insert(4, 'iso_adj_apt', 0)

In [12]:
#Filling in 3 new columns by 1 if Apt has specific features related to columns 
splited_list_of_num_rooms = [i.split(', ') for i in df['number_of_rooms']]

for i in range(len(splited_list_of_num_rooms)):
    if splited_list_of_num_rooms[i][0].isdigit():
        df.loc[i, 'number_of_rooms'] = splited_list_of_num_rooms[i][0]
        
        if len(splited_list_of_num_rooms[i]) > 1:
            if splited_list_of_num_rooms[i][1] == 'Изолированная':
                df.loc[i, 'isolated_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Смежная':
                df.loc[i, 'adjacent_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Оба варианта':
                df.loc[i, 'iso_adj_apt'] = 1

In [13]:
#Replacing nan values by the most frequent element 
df['metro'] = df['metro'].replace(to_replace=[None], value=df.loc[:, 'metro'].mode()[0])

In [14]:
#Adding new columns related to Metro feature
df.insert(6, 'time_to_get_metro', 0)
df.insert(7, 'by_car', 0)
df.insert(8, 'by_walk', 0)

In [15]:
#Check uniqueness of transport type and create lists for future distribution
splited_list_of_metro = [metro.split(' (') for metro in df['metro']]
lst_type_of_transport_metro = []
lst_time_to_get_metro = []
lst_name_of_metro = []
for i in splited_list_of_metro:
    lst_type_of_transport_metro.append(i[1].split()[-1].rstrip(')'))
    lst_time_to_get_metro.append(i[1].split()[0])
    lst_name_of_metro.append(i[0])
    
unique_lst = pd.Series(lst_type_of_transport_metro).unique()
unique_lst

array(['пешком', 'машине'], dtype=object)

In [16]:
#Filling in 2 new columns by 1 if Metro has specific features related to columns
#And adding amount of min to get metro in 1 column
for i in range(len(splited_list_of_metro)):
    df.loc[i, 'metro'] = lst_name_of_metro[i]
    df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]
    
    if lst_type_of_transport_metro[i] == 'пешком':
        df.loc[i, 'by_walk'] = 1
    else:
        df.loc[i, 'by_car'] = 1

/tmp/ipykernel_1583483/3928166692.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]


Bul's part

In [17]:
temp1=df['Цена']
rent_price_curr = temp1.apply(lambda x:x.split('/')[0].split())
rent_price = rent_price_curr.apply(lambda x:x[0])
# Цена аренды

In [18]:
currency = rent_price_curr.apply(lambda x:x[1]) # Валюта аренды

In [19]:
s3=temp1.apply(lambda x:x.split('Срок аренды - '))
stay_time=s3.apply(lambda x:x[1].split(',')[0]) # Виды сроков

In [20]:
deposite = temp1.apply(lambda x:x.split('Залог - '))
test1 = deposite.apply(lambda x:len(x))
bool_series1 = test1==2

deposite[bool_series1] = deposite[bool_series1].apply(lambda x:x[1].split(',')[0].split(' '))
deposite[~bool_series1] = '00'

deposite_value = deposite.apply(lambda x:x[0]) # Все залоги, если нету = то 0 строкой
deposite_currency = deposite.apply(lambda x:x[1]) # Все валюты, если нету = то 0 строкой

In [21]:
pre_pay = temp1.apply(lambda x:x.split('Предоплата ')[-1])

pre_pay_months = pre_pay.apply(lambda x:x.split(' ')[0]) 

pre_pay_bool = pre_pay.apply(lambda x:x.split(' ')[-1])
pre_pay_bool[(pre_pay_bool=='Длительный')|(pre_pay_bool=='месяцев')]='нету'
# Предоплата либо "мес", либо нету - заменил на "нету".

pre_pay_months = pre_pay_months.astype(float)
pre_pay_months[pre_pay_months>12]=0
# Количество месяцев

In [22]:
df = df.drop(['Цена'], axis=1) # Убераем цену

In [23]:
df.insert(12,'rent_price',rent_price)
df.insert(13,'rent_currency',currency)
df.insert(14,'stay_time',stay_time)
df.insert(15,'deposite',deposite_value)
df.insert(16,'deposite_currency',deposite_currency)
df.insert(17,'is_prepay',pre_pay_bool)
df.insert(18,'prepay_months',pre_pay_months)

In [24]:
df = df.drop(['Описание'], axis=1) # Убераем описание

In [25]:
df['rent_price']=df['rent_price'].astype(float)
df['deposite']=df['deposite'].astype(float)
# Переводит числовые во float

In [26]:
df = df.drop(['is_prepay'], axis=1) # Убераем логический признак
df['prepay_months'] = df['prepay_months'].astype(int) # Теперь если 0 - то нету Предоплаты, или количество месяцев предоплаты

In [27]:
df = df.rename(columns={'Ремонт': 'renovation'}) # Переименовываем колонку

In [28]:
df['renovation']=df['renovation'].fillna('Нету') # Пропуски = "Нету"

In [29]:
df = df.rename(columns={'Балкон': 'balcony'}) # Переименовываем колонку

In [30]:
df['balcony']=df['balcony'].fillna('Балкон (0), Лоджия (0)') # Пропуски = 'Балкон (0), Лоджия (0)'

In [31]:
# Здесь начинается вторая часть Второго релиза = конвертировать цены в Доллары, кодировать строки

In [32]:
df.loc[df['renovation'] == 'Нету', 'renovation'] = 'Без ремонта' # Повторяющиеся значения "Нету" и "Без ремонта" = теперь везде "Без ремонта"

In [33]:
renovation_mapping = {
    'Евроремонт': 3,
    'Дизайнерский': 4,
    'Косметический': 2,
    'Без ремонта':1,
}

df['renovation']=df['renovation'].map(renovation_mapping) # Сделали кодировку, где 1 = самый ужасный ремонт, 4 = самый лучший

In [34]:
convert_to_usd = {'руб.':0.011, '$':1, '€':1.07} # Словарь множителей от текущей валюты в Доллары
df['rent_price']=df.apply(lambda row: row['rent_price'] * convert_to_usd[row['rent_currency']], axis=1) # Переводим все цены в Доллары
df = df.drop(['rent_currency'], axis=1) # Убераем колонку валют

In [35]:
convert_to_usd_depo = {'руб.':0.011, '$':1, '€':1.07, '0':0.0} # Словарь уже для депозитов, там их кое-где нету валюты = 0
df['deposite'] = df.apply(lambda row: row['deposite'] * convert_to_usd_depo[row['deposite_currency']], axis=1) # Переводим все депозиты в Доллары
df = df.drop(['deposite_currency'], axis=1) # Убераем колонку валют

In [36]:
df['deposite']=df.apply(lambda row: row['deposite']/row['rent_price'], axis=1) # Переводим их в процентное соотношение от цены за аренду

In [37]:
stay_time_mapping = {
    'Длительный': 2,
    'На несколько месяцев': 1
}

df['stay_time']=df['stay_time'].map(stay_time_mapping) # Сделали кодировку, где 1 = на несколько месяцев, 2 = длительный срок

In [38]:
def extract_and_sum_numbers(s):
    numbers = re.findall(r'\d+', s)
    return sum(map(int, numbers))

df['balcony']=df['balcony'].apply(extract_and_sum_numbers)
# Используя regular expression я нахожу все числа и суммирую их
# В итоге Лоджия и Балкон по весу = у меня равны, а значения = это их кол-во

In [39]:
df = df.rename(columns={'balcony': 'balcony_count'}) 

In [40]:
df.iloc[:,12:18].dtypes # Все мои значения

rent_price       float64
stay_time          int64
deposite         float64
prepay_months      int64
renovation         int64
balcony_count      int64
dtype: object

Nastya's part

In [41]:
df['pets_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с животными' in x else (0 if isinstance(x, str) else np.nan))
mode_pets_okay = df['pets_okay'].mode()[0]
df['pets_okay'].fillna(mode_pets_okay, inplace=True)
print(df['pets_okay'])

0        1.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
23363    0.0
23364    0.0
23365    0.0
23366    0.0
23367    1.0
Name: pets_okay, Length: 23368, dtype: float64


/tmp/ipykernel_1583483/2283144236.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pets_okay'].fillna(mode_pets_okay, inplace=True)


In [42]:
percent_counts = df['pets_okay'].value_counts(normalize=True) * 100
# Print the percentage counts
print("Percentage of each value in 'pets_okay':")
print(percent_counts)

Percentage of each value in 'pets_okay':
pets_okay
0.0    69.453954
1.0    30.546046
Name: proportion, dtype: float64


In [43]:
df['kids_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с детьми' in x else (0 if isinstance(x, str) else np.nan))
mode_kids_okay = df['kids_okay'].mode()[0] # 🚧
df['kids_okay'].fillna(mode_kids_okay, inplace=True)
print(df['kids_okay'])

0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
        ... 
23363    1.0
23364    1.0
23365    1.0
23366    1.0
23367    1.0
Name: kids_okay, Length: 23368, dtype: float64


/tmp/ipykernel_1583483/1037702761.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['kids_okay'].fillna(mode_kids_okay, inplace=True)


In [44]:
percent_counts = df['kids_okay'].value_counts(normalize=True) * 100

# Print the percentage counts
print("Percentage of each value in 'kids_okay':")
print(percent_counts)

Percentage of each value in 'kids_okay':
kids_okay
1.0    98.977234
0.0     1.022766
Name: proportion, dtype: float64


In [45]:
def create_column(df, new_col_name, phrase):
    df[new_col_name] = df['Дополнительно'].apply(
        lambda x: 1 if isinstance(x, str) and phrase in x else (0 if isinstance(x, str) else np.nan)
    )
    # Fill NaN values with the mode
    mode_value = df[new_col_name].mode()[0]
    df[new_col_name].fillna(mode_value, inplace=True)
    return df

# Create new columns
df = create_column(df, 'furniture_in_the_room', 'Мебель в комнатах')
df = create_column(df, 'air_conditioner', 'Кондиционер')
df = create_column(df, 'dishwashing', 'Посудомоечная машина')
df = create_column(df, 'fridge', 'Холодильник')
df = create_column(df, 'internet', 'Интернет')

/tmp/ipykernel_1583483/390896744.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[new_col_name].fillna(mode_value, inplace=True)
/tmp/ipykernel_1583483/390896744.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [46]:
# Checking descriptive stats for each main feature
def calculate_percentage(df, column_name):
    percent_counts = df[column_name].value_counts(normalize=True) * 100
    print(f"Percentage of each value in '{column_name}':")
    print(percent_counts)

calculate_percentage(df, 'furniture_in_the_room')
calculate_percentage(df, 'air_conditioner')
calculate_percentage(df, 'dishwashing')
# Doing reality check - these columns should largely have 1 
calculate_percentage(df, 'fridge')
calculate_percentage(df, 'internet')


print(df[['furniture_in_the_room', 'air_conditioner', 'dishwashing', 'fridge', 'internet']])

Percentage of each value in 'furniture_in_the_room':
furniture_in_the_room
1.0    92.643786
0.0     7.356214
Name: proportion, dtype: float64
Percentage of each value in 'air_conditioner':
air_conditioner
0.0    62.358781
1.0    37.641219
Name: proportion, dtype: float64
Percentage of each value in 'dishwashing':
dishwashing
0.0    66.48836
1.0    33.51164
Name: proportion, dtype: float64
Percentage of each value in 'fridge':
fridge
1.0    88.124786
0.0    11.875214
Name: proportion, dtype: float64
Percentage of each value in 'internet':
internet
1.0    80.498973
0.0    19.501027
Name: proportion, dtype: float64
       furniture_in_the_room  air_conditioner  dishwashing  fridge  internet
0                        1.0              1.0          1.0     1.0       1.0
1                        1.0              1.0          1.0     1.0       1.0
2                        1.0              1.0          1.0     1.0       1.0
3                        1.0              1.0          1.0     1.0      

In [47]:
df['only_street_view'] = df['Окна'].apply(
    lambda x: 1 if isinstance(x, str) and 'На улицу' in x else (0 if isinstance(x, str) else np.nan)
)
mode_only_street_view = df['only_street_view'].mode()[0]
df['only_street_view'].fillna(mode_only_street_view, inplace=True)
percent_counts = df['only_street_view'].value_counts(normalize=True) * 100

print("Percentage of each value in 'only_street_view':")
print(percent_counts)

Percentage of each value in 'only_street_view':
only_street_view
0.0    74.815988
1.0    25.184012
Name: proportion, dtype: float64


/tmp/ipykernel_1583483/3602654623.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['only_street_view'].fillna(mode_only_street_view, inplace=True)


In [48]:
#Getting rid of columns 
df = df.drop(['Окна'], axis=1)
df = df.drop(['Можно с детьми/животными'], axis=1)
df = df.drop(['Дополнительно'], axis=1)
df = df.drop(['Название ЖК'], axis=1)
df = df.drop(['Серия дома'], axis=1)
df = df.drop(['Ссылка на объявление'], axis=1)


In [49]:
df.to_csv('../data/data.csv')